In [2]:
import json


with open("reddit_AskNYC_washingtondc.json", "r") as f:
    reddit_data = json.load(f)


print(len(reddit_data))  # Should be 6274 elements
print(json.dumps(reddit_data[list(reddit_data.keys())[0]], indent=2))




6158
{
  "subreddit": "AskNYC",
  "author": "Puzzleheaded-Farm431",
  "title": "Paid family leave",
  "selftext": "Hi all, I work for a small company with very few employees and no form of office really just a fabricating shop for local 28,\nI will be applying for paid family leave once my wife is due. I feel my boss  won\u2019t fill me in on how the application process works. How can I go about this? Do I need my boss to fill things out? Can I just do it online ?",
  "year": 2024,
  "month": 1,
  "score": 1,
  "num_comments": 1,
  "comments": []
}


In [3]:
from langchain.text_splitter import RecursiveJsonSplitter


reddit_list = list(reddit_data.values())  

splitter = RecursiveJsonSplitter(max_chunk_size=500)  
documents = splitter.create_documents(reddit_list)  

print(f"Generated {len(documents)} documents")


Generated 9028 documents


In [4]:
!pip install -U langchain-community


In [5]:
!pip install -U sentence-transformers torch torchvision torchaudio


  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached nvidia_cusparselt_cu12-0.6.2-py3-none-manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached triton-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
  Using cached safetensors-0.5.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached sentence_transformers-3.4.1-py3-none-any.whl (275 kB)
Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl (766.7 MB)
Using cached nvidia_cusparselt_cu12-0.6.2-py3-none-manylinux2014_x86_64.whl (150.1 MB)
Using cached triton-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (253.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 31.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 97.7 MB/s

In [6]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}  
encode_kwargs = {"normalize_embeddings": True}

embeddings_model = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Create embeddings
embedded_documents = embeddings_model.embed_documents([doc.page_content for doc in documents])


/tmp/ipykernel_16012/2810933727.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceBgeEmbeddings(


In [8]:
print(doc.page_content)  # Check if the document content is available


NameError: name 'doc' is not defined

In [1]:
metadatas = []
for doc in documents:
    metadata = {
        "subreddit": doc.metadata.get("subreddit_x"),  # Changed to match the column names
        "num_comments": doc.metadata.get("num_comments"),
        "score": doc.metadata.get("score_x"),  # Changed to match the column names
        "year": doc.metadata.get("yyyy_x"),  # Changed to match the column names
        "author": doc.metadata.get("author_x"),  # Changed to match the column names
        "id": doc.metadata.get("id_x"),  # Changed to match the column names
    }
    metadatas.append(metadata)


NameError: name 'documents' is not defined

In [32]:
!conda install -c conda-forge faiss-cpu -y


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Channels:
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs:
    - faiss-cpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.4.1              |       h7b32b05_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following NEW packages will be INSTALLED:

  faiss              conda-forge/linux-64::faiss-1.9.0-py310he38f182_0_cpu 
  faiss-cpu          conda-forge/linux-64::faiss-cpu-1.9.0-h718b53a_0 
  libfaiss           conda-forge/linux-64::l

In [33]:
import numpy as np
import faiss
embedding_vectors = np.array(embedded_documents)

# Create a FAISS index
index = faiss.IndexFlatL2(embedding_vectors.shape[1])  # Using L2 distance for similarity search
index.add(embedding_vectors)

In [35]:
import os 
faiss_index_dir = "faiss_index"
os.makedirs(faiss_index_dir, exist_ok=True)

# Save the index to a file
faiss.write_index(index, os.path.join(faiss_index_dir, "faiss_index.index"))


In [36]:
with open(os.path.join(faiss_index_dir, "metadata.json"), "w") as f:
    json.dump(metadatas, f, indent=4)

In [37]:
print(f"Metadata sample: {metadatas[:5]}")


Metadata sample: [{'subreddit': None, 'num_comments': None, 'score': None, 'year': None, 'author': None, 'id': None}, {'subreddit': None, 'num_comments': None, 'score': None, 'year': None, 'author': None, 'id': None}, {'subreddit': None, 'num_comments': None, 'score': None, 'year': None, 'author': None, 'id': None}, {'subreddit': None, 'num_comments': None, 'score': None, 'year': None, 'author': None, 'id': None}, {'subreddit': None, 'num_comments': None, 'score': None, 'year': None, 'author': None, 'id': None}]
